In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from joblib import Parallel, delayed
import time

from skimage.transform import rescale

In [2]:
stack = 'MD594'
show_labels = True

In [15]:
labelingViz_root = '/oasis/projects/nsf/csd395/yuncong/CSHL_annotationsViz'
labelingViz_dir = os.path.join(labelingViz_root, stack)
os.system('rm -r %s' % labelingViz_dir)
create_if_not_exists(labelingViz_dir)

sp5
RtTg
12N

Tzsp5Pn

AP
AP
6N
Tz12N
12N
sp5
Pn12N
sp5
12N


RtTg
RtTg

Pn
RtTg
RtTg

sp5
Tz7n
sp5
sp5AP
sp5
12N
AP12N
RtTg
RtTg
Pn
Pn
sp5
Tz

Tz
Tz
R
12N
AP

sp5
12N
12N
sp5sp512N

sp5
sp5
RtTg

12N
RtTg
R
R
Pn
Tzsp5
sp5


R
sp5
12N
Pn
12N
sp5
R
Pn
RtTg
RtTg
RtTg

Pnsp512N
12N
RtTg
12N
R
LRt
sp5
12N
RtTg
TzPn
Pn
12N
sp5


AP
AP
Pn
R
RtTg
7N
TzR
Pn

7nPnsp5
12N
Tz
12N
RtTg
RtTg
7nRtTg
Pn
VLL
RtTg
Pn7n


sp5R
RtTg
AP
Pn
Pn

Pn
Pn
LC
Pn

sp5
Pn
Tz
RtTg
12N
R
TzPnPn
7n
7n
7n
7nTz
RtTg
Tz

12N
Pn
sp5
RtTg


Tz

RtTg
Amb
sp5

RtTg
Pn
RtTg
LC
AP
7nTzPn
sp5
Tz
RtTg
Pn
7nsp5
RtTg
Pn
sp5
LCsp5
7n
R


7n12N
RtTg
sp5
Tz

5N
Pn
Tz
LC
7nsp5
RtTg
Pn
LC

R
12N
LCRtTg
Pn
LRt
VLLRtTg

7n

RtTg
Pn
Tz
7n
Pn
RtTg
sp5

sp5
sp5
7N

sp5
7n
sp5
Pn
7N
PnRtTg
sp5
Tz
Pn
Tz7n
7nLC


'/oasis/projects/nsf/csd395/yuncong/CSHL_annotationsViz/MD594'

7n
7nsp5
RtTg
sp5
Pn

sp5
RtTg
RtTg
Tz
sp5

7N
sp5

RtTg
7N
LC
VLLTz
7n7N
sp5

LC
RtTg
sp5
VLL5N
Pn
7N
Pn
7N

RtTg

Pn
7nLC
7n
7N
5N

LRt
sp5
Pn
VLLVLL7n
12N
Pn
VLL
7n
sp5
Pn
LRt
7n
7N
LC
VLL

sp5
sp5
sp5

Pn
sp5
7N
VLL7N
Amb
Pn
7N

7n
7n
5N
LCLC
7n
sp5
RtTg
Pn

Pn
sp5
VLLVLL7n
Amb
Amb
LRt

7N
sp5
LC
Pn
VLL7n
VLL5N


sp5
sp5
sp5
7N
7n
VLLLRt
7N
7n
sp5

LRt
7n
7n
5N
5N
5N
Pn
sp5

7N
VLL
7n
5N
7n
7N
Amb
Amb
LRt
LRt
LRt
VLLRtTg
7n
Pn

sp5
sp5
LRt
Amb
sp5
sp5
7N
7N
7N

7N
Amb
VLL7n
Pn
5N
7N
sp5
5N
5N
Pn
VLL7n
Pn
sp5

Amb
LRt
LRt
VLL5N
LRt
LRt
VLL
LRt
sp5
VLL5N
7n
sp5
7N
7N

7N
7N

7n
7N
5N

LRt
Amb
5N
VLLPn
7n
VLL7n
sp5
sp5
LRt
7n
7N
sp5
LRt

VLL

sp5
5N
5N
7N
sp5
VLL5N
7N
7n
7n
7n
5N
LRt
7N
5N

LRt
Amb
Amb
sp5
LRt
7N
sp5
LRt
7n
7N
sp5
5N
7N
5N
7N
sp5
5N
7N
5N
Pn
LRt
LRt
7N
7N
VLL7N

7n
sp5
5N
LRt
7N


In [16]:
downscale_factor = 8

In [17]:
dm = DataManager(stack=stack)

In [18]:
# parallel

In [19]:
def f(sec):
    
    dm.set_slice(sec)
    
    ret = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    if ret is not None:
        user, ts, _, annotations = ret
        
    ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
    if ret_autoAnno is not None:
        user2, ts2, _, annotations_autoAnno = ret_autoAnno        

#     ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        return
        
    img = imread(os.path.join(os.environ['DATA_DIR'], '%(stack)s_lossless_aligned_cropped_downscaled', 
                  '%(stack)s_%(sec)04d_lossless_aligned_cropped_downscaled.jpg') % {'stack': stack, 'sec': sec})

#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    viz = img[::downscale_factor, ::downscale_factor].copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)

            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            print ann['label']

            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
            
    cv2.imwrite(output_fn, viz[..., ::-1])

In [20]:
first_bs_sec, last_bs_sec = section_range_lookup[stack]

In [21]:
import time
t = time.time()

_ = Parallel(n_jobs=16)(delayed(f)(sec) for sec in range(first_bs_sec, last_bs_sec+1))

# simultaneous write seems fine
sys.stderr.write('generate visualization: %.2f seconds\n' % (time.time() - t)) # ~ 60s/150 sections; ~200s/stack

sp5
5N
7n
5N
7n
7n
7n
7n
7n
VLL
VLL
7n
VLL
7n
VLL
7n
VLLsp5
VLL
sp5
5N
5N
VLL
5N
5N
7n
7n
Amb
7n
Amb
7n
Amb

VLL
sp5
VLL
sp5
sp5
sp5
sp5
VLL
sp5
Amb
sp5
sp5
sp5
Amb
sp5
sp5
5N7n7N
VLL
VLL
7nVLL
7N
5N
sp5
5N
5N
5N
sp5
LRt
7n

7n7N
7N

7N
VLL7N
5N
LRt
LRt
LRt
5N
7N

sp5
VLL

7n7n7N
7n
VLLLRt
7N
7N
7N
LRt
Pn
sp5
VLL
sp5
5N


5N

7n

7N
7nVLLVLL7N
sp5LRt
7nLCVLL
5N
5N
VLL
5N
5N
7n
VLL


Pn

7N


LCsp5
sp5
LCVLL
LRt
sp5

Amb
7n
7n
7n5N
Pn
sp5
7n

VLL
VLL

7N
7N
5N
7n
sp5
sp5
Amb

7N
7nLRt
sp5
7n
7N
7N
7n
LCLCLRt
Amb
5N
5N
sp5
Amb
Pn

7N
LRt
sp5
7nLCsp5


7N
sp5
LRt
LRt
5N
sp5
Tzsp5
Pn
7N
LRt


R
7n
7n
Tz5N
7N
7N
LRt
5N

LRt
LCPn
7N
sp5
7n
LRt
sp5
sp5

LRt
Pn
Pn
7N
LRt
7n
7N

RtTg
Pn
7N
sp5
7N
R
R
LC
7N
TzTzPn
7N
sp5
Pn
7n
LCRtTg
Pn
LRt
Pn
LRt
RtTg
7n
LC

TzPn
R


generate visualization: 53.76 seconds
username yuncong does not have any annotations for current section 97 
username yuncong does not have any annotations for current section 93 
username yuncong does not have any annotations for current section 94 
username yuncong does not have any annotations for current section 95 
username yuncong does not have any annotations for current section 99 
username yuncong does not have any annotations for current section 101 
username autoAnnotate does not have any annotations for current section 98 
username yuncong does not have any annotations for current section 105 
username yuncong does not have any annotations for current section 103 
username autoAnnotate does not have any annotations for current section 100 
username yuncong does not have any annotations for current section 107 
username autoAnnotate does not have any annotations for current section 104 
username autoAnnotate does not have any annotations for current section 96 
username auto

Tzsp5

LC7n7N
RtTg
7N
7N
sp5

7n
7n

TzRtTg

LRt
7n


Pn
TzPn
Pn
R
7n
sp5
sp5
7n

Pn
sp5
7N
sp5
7n
sp5
LC
RtTg
LRt
RtTg
sp5
R
R
sp5
7n
APRtTg
Pn
LRt
sp5
LRt

7n
TzTz7N
R
RtTg
RtTg
R
sp5

Pn
Tz7N
LRt
7N
7n
sp5


Pn
RtTg
7N
7N
RtTg
RtTg
Pn
Pn
Pn
7N
Pn
sp5
RtTg
sp5
7n
LRt
7N
Pn
Pn
Pn
Pn
RtTg

sp5
RtTg
Pn
TzR
Pn
R
sp5
TzPn
TzTzTzsp512N
RtTg
R
TzRtTg

LRt
TzRtTg
RtTg

LRt




sp5
12N
RtTg

Tzsp5
7N

Pn
Pn
7n
Tz7n
sp5
sp5
6N
sp5sp5
Pn
sp5

R
Pn
sp5
APsp5sp5

sp5
R
6N
12N

12NAPR
sp5
RtTg
RtTg
R


R
7n
R
12N
12N
Pn
12N


Pn
R
Pn
TzPn
Pn
6N
RtTg
sp5
RtTg
Pn
Pn
RtTg
AP
AP
Pn
RtTg
Pn
AP
RtTg
RtTg
12N
Pn
R
Pn
RtTg
RtTg
APR
R
RtTg
APRtTg

sp5
AP12N
Pn
12NRtTg
sp5APAP
RtTg
RtTg
12N

APPn
R

sp5
RtTg

Pn



RtTg
Pn
Pn
sp5
Pn

RtTg
Pn
12N
TzAPsp5
Tz12N
RtTg
RtTg
12N
Pn

In [ ]:
########## serial version ##########

first_bs_sec, last_bs_sec = section_range_lookup[stack]

for sec in range(first_bs_sec, last_bs_sec+1):
# for sec in range(181, 262+1):
# for sec in [135]:

    dm.set_slice(sec)
    
    ret = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    if ret is not None:
        user, ts, _, annotations = ret
        
#     ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
#     if ret_autoAnno is not None:
#         user2, ts2, _, annotations_autoAnno = ret_autoAnno        
    ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        continue
    
    print '\n'
    print sec
    
    img = imread(os.path.join(os.environ['DATA_DIR'], '%(stack)s_lossless_aligned_cropped_downscaled', 
                  '%(stack)s_%(sec)04d_lossless_aligned_cropped_downscaled.jpg') % {'stack': stack, 'sec': sec})

#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    viz = img[::downscale_factor, ::downscale_factor].copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)

            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            print ann['label']

            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
        
    cv2.imwrite(output_fn, viz[..., ::-1])

In [ ]:
# For MICCAI paper (image, scoremap, auto annotation contours)

In [32]:
first_bs_sec, last_bs_sec = section_range_lookup[stack]

# for sec in range(first_bs_sec, last_bs_sec+1):
for sec in [141]:

    dm.set_slice(sec)
        
    ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
    if ret_autoAnno is not None:
        user2, ts2, _, annotations_autoAnno = ret_autoAnno
        
    if ret is None and ret_autoAnno is None:
        continue
    
    print sec
    
    
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]
    
    scoremaps_rootdir = '/home/yuncong/csd395/CSHL_scoremaps_lossless/'
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)

    l = '7N'
    
    scoremap_whole = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                           '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                           {'stack': stack, 'sec': sec, 'label': l}))
    # except:
    #     sys.stderr.write('No scoremap of %s exists\n' % (l))
    #     continue


    dataset = stack + '_' + '%04d'%sec + '_roi1'

    interpolation_xmin, interpolation_xmax, \
    interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                     '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                    {'dataset': dataset, 'label': l})).astype(np.int)

    dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
    dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                            interpolation_xmin:interpolation_xmax+1] = scoremap_whole

    scoremap = dense_scoremap_lossless[::8, ::8]
    scoremap_height, scoremap_width = scoremap.shape[:2]

    scoremap_layer_viz = img_as_ubyte(plt.cm.hot(scoremap)[..., :3])
    scoremap_viz = img_as_ubyte(alpha_blending(scoremap_layer_viz,
                                               cropped_img,
                                               .3, 1)[..., :3])
    

#     viz = img_as_ubyte(rescale(scoremap_viz, 1./downscale_factor))
#     viz = scoremap_viz[::downscale_factor, ::downscale_factor]
                
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            if ann['label'] == l:
            
                vertices = np.array(ann['vertices']) / downscale_factor
                cv2.polylines(scoremap_viz, [vertices.astype(np.int)], True, (255,0,0), 2)

    #             print ann['label']


141


In [33]:
display(scoremap_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.jpg